In [1]:
#Author Alex J. Yang, alexjieyang@outlook.com
import numpy as np
import pandas as pd

In [2]:
cd E:\Patent

E:\NETDATA\Patent


# Load data

In [3]:
df1 = pd.read_csv('Inventor\g_patent_inventor.csv.zip')

In [4]:
df1.columns

Index(['patent_id', 'inventor_id', 'location_id', 'date'], dtype='object')

In [5]:
df1.dropna(subset = 'location_id' , inplace = True)

In [6]:
df2 = df1.groupby(['inventor_id'])['patent_id'].count().reset_index()
df2.columns = ['inventor_id' , 'Productivity']

In [9]:
df2 = df2[df2['Productivity']>=2].copy()
df1 = df1[df1['inventor_id'].isin(df2['inventor_id'])].copy()

In [10]:
len(df1)

20140278

In [11]:
len(df2)

2283542

# Count #unique locations for each author

In [12]:
dff = df1.groupby('inventor_id')['location_id'].nunique().reset_index()
dff.columns = ['inventor_id','#locations']

I choose the authors with mutiple insitutions, as these authors move at least once

In [13]:
dff = dff[dff['#locations']>1]

In [14]:
df1 = df1[df1['inventor_id'].isin(dff['inventor_id'])].copy()

# Sort papers based on the inventor_id and Publication date

In [16]:
df1.sort_values(['inventor_id','date'],inplace=True)

## calculate the moving point

In this part, I use the  location_id and formerlocation_id to determine the moving point.
For a scientist, in her career, if the current location_id is different from the former one, I consider it as the moving point.

Note that this may raise error when it comes to the junction of two authors.
To address this issue, I use the inventor_id and formerinventor_id to determine if this is the author's last publication in her careers,
and I remove the moving points in the last publications for each author

In [18]:
df1['formerlocation_id'] = np.concatenate( [[11944219.0], df1['location_id'].values[:-1]])
df1['ifMove'] = 0
df1.loc[df1['location_id'] != df1['formerlocation_id'] , 'ifMove']=1

remove the moving points in the last publications for each author

In [20]:
# df1['formerinventor_id'] = np.concatenate( [ [0] , df1['inventor_id'].values[:-1]])
# df1['ifLastPub'] = df1['inventor_id'] - df1['formerinventor_id']
# df1['ifLastPub'] = df1['ifLastPub'].apply(lambda x:0 if x==0 else 1)

df1['formerinventor_id'] = np.concatenate( [ [0] , df1['inventor_id'].values[:-1]])
df1['ifFirstPub'] = 0
df1.loc[df1['inventor_id'] != df1['formerinventor_id'] , 'ifFirstPub']=1

In [21]:
df1 = df1[df1['ifFirstPub']==0]
del df1['ifFirstPub']
del df1['formerinventor_id']
df1.index = range(len(df1))

In [27]:
df2 = df1[df1['ifMove']==1]
df2.to_csv(r'Inventor\Move\mobility.csv.zip' , index = False)

## Dertermine if mobility

In [43]:
Move_count = df1.groupby('inventor_id')['ifMove'].sum().reset_index()

In [44]:
df1['Year'] = df1['date'].apply(lambda x:int(x[:4]))

In [45]:
df11 = df1[df1['inventor_id'].isin(Move_count.loc[Move_count['ifMove']==1 , 'inventor_id'].values)]
df12 = df1[df1['inventor_id'].isin(Move_count.loc[Move_count['ifMove']>1 , 'inventor_id'].values)]

In [46]:
def filter_move_points(group):
    move_points = group[group['ifMove'] == 1].index.tolist()
    valid_moves = []

    for point in move_points:
        # Check if there are at least 3 years before and after the move point
        before = group[(group['Year'] >= group.loc[point, 'Year'] - 2) & (group['date'] < group.loc[point, 'date'])]
        after = group[(group['date'] > group.loc[point, 'date']) & (group['Year'] <= group.loc[point, 'Year'] + 2)]

        # If the author didn't move for 3 years before and after the move point, consider it a valid move
        if len(before[before['ifMove'] == 1]) == 0 and len(after[after['ifMove'] == 1]) == 0:
            valid_moves.append(point)

    # Return indices of move points that meet the criteria
    return valid_moves

In [47]:
valid_move_indices = df12.groupby('inventor_id').apply(filter_move_points).explode().values

In [48]:
dfmove1 = df11[df11['ifMove']==1]

In [49]:
dfmove2 = df1.loc[pd.DataFrame(valid_move_indices).dropna()[0].values,:]

In [50]:
dfmove = pd.concat([dfmove1 , dfmove2])

In [51]:
dfmove

,patent_id,inventor_id,location_id,date,formerlocation_id,ifMove,Year
0,4462769,fl:(l_ln:schibbye-1,4027393c-16c8-11ed-9b5f-1234bde3cd05,1984-07-31,51a5a1ae-16c8-11ed-9b5f-1234bde3cd05,1,1984
4,11645378,fl:._ln:sharanabasappa-1,93f56e17-49af-11ed-9879-1234bde3cd05,2023-05-09,68a4ea01-16c8-11ed-9b5f-1234bde3cd05,1,2023
8,7744238,fl:a-_ln:chang-3,b5c9a039-16c7-11ed-9b5f-1234bde3cd05,2010-06-29,907f9ab3-16c8-11ed-9b5f-1234bde3cd05,1,2010
28,4724885,fl:a-_ln:chang-5,b5c9a039-16c7-11ed-9b5f-1234bde3cd05,1988-02-16,8fb65d2f-16c8-11ed-9b5f-1234bde3cd05,1,1988
33,9074409,fl:a-_ln:chen-5,90e86893-16c8-11ed-9b5f-1234bde3cd05,2015-07-07,90a89965-16c8-11ed-9b5f-1234bde3cd05,1,2015
...,...,...,...,...,...,...,...
13709768,4156758,fl:zy_ln:tomczuk-1,a171c96c-16c8-11ed-9b5f-1234bde3cd05,1979-05-29,ebac7b62-16c7-11ed-9b5f-1234bde3cd05,1,1979
13709773,5282937,fl:zy_ln:tomczuk-1,e7bcffcf-16c7-11ed-9b5f-1234bde3cd05,1994-02-01,a171c96c-16c8-11ed-9b5f-1234bde3cd05,1,1994
13709782,7011736,fl:zy_ln:tomczuk-1,e409932f-16c7-11ed-9b5f-1234bde3cd05,2006-03-14,e7bcffcf-16c7-11ed-9b5f-1234bde3cd05,1,2006
13709785,4053842,fl:zy_ln:turski-1,05e39536-16c8-11ed-9b5f-1234bde3cd05,1977-10-11,0d139b93-16c8-11ed-9b5f-1234bde3cd05,1,1977


In [52]:
dfmove.to_csv(,index = False)